In [1]:
###%run dihiggs_dataset.ipynb TODO: add something 

import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F   # NOTE: I don't think this is used
import torch.autograd as autograd
import torch

print(torch.cuda.is_available())

False


In [ ]:
class opt():   # Class used for optimizers in the future. Defines all variables and stuff needed.
    opt.save_weights = False
    n_epochs = 20000   # an epoch is the number of times it works through the entire training set
    batch_size = 4000   # the training set is broken up into batches, 
                        # and the average loss is used from a given batch for back propagation
    lr = 0.001   # learning rate (how much to change based on error)
    b1 = 0.9   # Used for Adam. Exponential decay rate for the first moment
    b2 = 0.999   # Used for Adam. Exponential decay rate for the second moment estimates (gradient squared)
    input_size = ?????
    data_file = "/data"
    config_dir = "."
    save_location = config_dir + data_file

    #n_cpu = 2    not used rn


cuda = True if torch.cuda.is_available() else False


class Classifier(nn.Module):
    """
    classifier layers
    """
    def __init__(self):
        super(Discriminator, self).__init__()   # Just uses the module constructor with name Discriminator 

        self.model = nn.Sequential(
            nn.Linear(input_size, 3),   # first layer
            nn.LeakyReLU(0.2, inplace=True),   # apply leaky relu to layer
        )

    def forward(self, input):
        """
        applies model to input and attempts to classify
        """
        output = self.model(input)   # Classifies the input (at location) as gg (0) qqbar (1) or other (2)
        return output


# ******* OUT OF CLASSES NOW ************

# Initialize generator and discriminator
classifier = Classifier()

if cuda:
    classifier.cuda()

    

# # Configure data loader - CHANGE
os.makedirs("./data/", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    DiHiggsSignalMCDataset('./DiHiggs Data', generator_level = False),
    batch_size=opt.batch_size,
    shuffle=True,
)
print('done')

In [ ]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2)) 
criterion = nn.MSELoss()   # mean squared error loss

batches_done = 0   # Counter for batches
for epoch in range(opt.n_epochs):   # Loop through all epochs
    for target, x in enumerate(dataloader): # x is in dataloader (a batch I think). target should be the correct values of the batches

        # Configure input
        input = Variable(x.type(Tensor))   # Variable is a wrapper for the Tensor x was just made into

        # ---------------------
        #  Train Classifier
        # ---------------------

        optimizer.zero_grad()   # Make gradients zero so they don't accumulate
        
        # apply nn to input
        output = classifier(input)

        # Calculate loss 
        loss = criterion(output, target) # -torch.mean(output) + torch.mean(y) # may need to change datatype of y

        loss.backward()   # Do back propagation 
        optimizer.step()   # Update parameters based on gradients for individuals


    if epoch % 10 == 0:
        np.save('{file_name}/gen_data/{num_batches}.npy'.format({file_name = opt.save_location}, \
            num_batches=batches_done), d_loss.cpu().detach().numpy()

if opt.save_weights:
    torch.save(classifier.model, opt.config_dir + "/singleLayerModel.pt")


